In [1]:
import os
from astroNN.apogee import allstar, visit_spectra, combined_spectra, gap_delete, apogee_continuum, bitmask_boolean
from astroNN.models import load_folder
from astroNN.apogee.chips import wavelength_solution, chips_split

from astropy.io import fits
from astropy.wcs import WCS
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import traceback
import h5py as h5
import time

%matplotlib inline
import matplotlib.ticker as ticker

In [2]:
pd.options.display.max_columns = 500

In [3]:
geir_path = "/geir_data/scr/petershi" #should save work here 
sdssv_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star" #path for SDSSV spectra
sdssiv_path = "/yngve_data/sdss/apogee/dr17/apogee/spectro/redux/dr17/stars/apo25m"

xmatch_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/aux_xmatch/ipl-1_dr17_lite_230517_byYS.fits"
model_path = "/geir_data/scr/petershi/models/retrain 2" #/geir_data/scr/petershi/models/astroNN_APOGEE_VAC/astroNN_0512_run002"

# Functions

In [4]:
def getSpectrum(star, ind, mask_value):

    spec = np.array(star["FLUX"])
    spec_err = np.array(star["FLUX_ERR"])
    bitmask = np.array(star["BITMASK"])

    target_bit = [0, 1, 2, 3, 4, 5, 6, 7, 12]

    try:
        _normspec, _spec_err = apogee_continuum(spec, spec_err, 
                                                bitmask=bitmask, 
                                                target_bit=target_bit, 
                                                dr=17, mask_value=mask_value)

        return (_normspec, _spec_err)

    except np.linalg.LinAlgError: # for linalgerror
        print("LinAlgError encountered for " + str(ind))
        return None

In [5]:
h5_path = os.path.join(geir_path, "serialized", "SDSS-V Stars.hdf5")

In [6]:
hf = h5.File(h5_path, 'r')

In [7]:
keys = list(hf.keys())

# Compile Spectra

In [8]:
spec_l = []
ind_l = []

for key in tqdm(keys):
    star = hf[key]

    specs = getSpectrum(star, key, 1)

    if specs is not None:
        normspec = specs[0]
        spec_l.append(normspec[0])

        ind_l.append(key)


  0%|          | 0/152024 [00:00<?, ?it/s]/tmp/ipykernel_23266/2556626446.py:3: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
/tmp/ipykernel_23266/2556626446.py:4: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
/tmp/ipykernel_23266/2556626446.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  0%|          | 662/152024 [00:03<12:42, 198.44it/s]

LinAlgError encountered for 27021597780338713

Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.


  1%|          | 789/152024 [00:03<12:31, 201.25it/s]

LinAlgError encountered for 27021597781064796
Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.



  1%|          | 900/152024 [00:04<12:01, 209.49it/s]

...

100%|█████████▉| 151887/152024 [08:34<00:00, 334.16it/s]

LinAlgError encountered for 5407850965
Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.

LinAlgError encountered for 5408254000

Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.


100%|█████████▉| 151989/152024 [08:34<00:00, 333.78it/s]

LinAlgError encountered for 5408383502
Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.



100%|██████████| 152024/152024 [08:34<00:00, 295.28it/s]


In [9]:
spec_l

[array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
 array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
 array([0.97309786, 0.9728667 , 0.98240757, ..., 0.99395514, 0.98564   ,
        0.96763045], dtype=float32),
 array([0.8286081 , 0.8535773 , 0.91142625, ..., 0.97486883, 0.95731026,
        0.9303909 ], dtype=float32),
 array([0.68368953, 0.7814658 , 0.8984029 , ..., 0.98040974, 0.9769394 ,
        0.948203  ], dtype=float32),
 array([0.8078921 , 0.81392515, 0.8978491 , ..., 0.9806892 , 0.9616927 ,
        0.99145615], dtype=float32),
 array([0.9471361 , 0.92610765, 0.94934857, ..., 0.9455386 , 0.9607191 ,
        0.88948053], dtype=float32),
 array([1.0392305 , 1.0343573 , 0.99267787, ..., 1.0776999 , 1.1356446 ,
        1.3196684 ], dtype=float32),
 array([0.8605639 , 0.94387203, 0.9734334 , ..., 0.9281575 , 0.99733996,
        0.9816088 ], dtype=float32),
 array([1.0008094 , 1.0311785 , 1.0555774 , ..., 0.98089224, 1.0181981 ,
        1.0018282 ], dtype=float32),
 array([0.83

Load Model

In [10]:
model = load_folder(model_path)

2023-07-05 22:26:29.627113: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11385 MB memory:  -> device: 0, name: TITAN Xp, pci bus id: 0000:04:00.0, compute capability: 6.1
/home/petershi/miniconda3/envs/mainenv/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.


Loaded astroNN model, model type: Bayesian Convolutional Neural Network -> ApogeeBCNNCensored


In [11]:
cols = model.targetname

In [12]:
preds = model.predict(spec_l)

Prediction progress:   0%|          | 64/145595 [00:08<5:11:32,  7.79sample/s, Monte-Carlo=100]2023-07-05 22:27:05.560332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Prediction progress: 100%|██████████| 145595/145595 [03:13<00:00, 141.00sample/s, Monte-Carlo=100]2023-07-05 22:30:10.683845: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Prediction progress: 100%|██████████| 145595/145595 [03:14<00:00, 750.20sample/s, Monte-Carlo=100]
/home/petershi/miniconda3/envs/mainenv/lib/python3.10/si

In [13]:
points = pd.DataFrame(preds[0], columns = cols)
errs = pd.DataFrame(preds[1]["total"], columns = cols)

points["ID"] = ind_l
errs["ID"] = ind_l

In [14]:
points.to_csv(os.path.join(geir_path, "serialized", "SDSS-V (ALL) points - retrain.csv"), index = False)
errs.to_csv(os.path.join(geir_path, "serialized", "SDSS-V (ALL) errors - retrain.csv"), index = False)

In [15]:
points

,teff,logg,C,CI,N,O,Na,Mg,Al,Si,P,S,K,Ca,Ti,TiII,V,Cr,Mn,Fe,Co,Ni,ID
0,4764.867676,3.367551,-1.218930,-1.235952,-1.356836,-1.102028,-1.166686,-1.332733,-1.575550,-1.103627,-1.352423,-0.976301,-1.079750,-1.305616,-1.460817,-1.028681,-1.281695,-1.647069,-1.647646,-1.322813,-1.189421,-1.268859,18014398509500000
1,4552.935547,2.939479,-0.664613,-0.778846,-0.668716,-0.659505,-0.768945,-0.924251,-1.053406,-0.758479,-0.884668,-0.518620,-0.690300,-0.702539,-0.907848,-0.776158,-0.840451,-0.840649,-0.896379,-0.830341,-0.603020,-0.739057,18014398509600000
2,4764.396484,2.571474,-0.303738,-0.313190,-0.064251,-0.116744,-0.237923,-0.126434,-0.111297,-0.139869,-0.251428,-0.122943,-0.103279,-0.184283,-0.193996,-0.152923,-0.349987,-0.233588,-0.232759,-0.211447,-0.163911,-0.191659,18014398509700000
3,4278.553711,1.734360,-0.124294,-0.187943,0.317653,0.003980,0.042598,-0.033130,-0.059683,-0.014835,-0.058894,0.020275,0.016338,-0.047207,0.011369,0.087633,-0.195979,0.031799,0.054328,-0.009606,0.091774,-0.002431,27021597767422950
4,3903.183350,1.057343,-0.280488,-0.330757,-0.061260,-0.175777,-0.221626,-0.184841,-0.269572,-0.252141,-0.280279,-0.132780,-0.161101,-0.283988,-0.181975,-0.198253,-0.369517,-0.252026,-0.168641,-0.262534,-0.147775,-0.238357,27021597767427884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145590,5610.926270,3.795910,-1.302636,-1.432425,-0.593905,-0.991536,-1.331727,-1.312761,-1.499395,-1.095954,-1.345334,-1.023950,-1.214589,-1.297424,-1.711647,-1.284057,-1.383361,-1.567217,-1.630185,-1.358305,-1.174365,-1.384064,661746642
145591,5850.075195,3.316058,-1.431771,-1.457500,-0.466899,-1.073747,-1.327014,-1.337163,-1.403619,-1.102864,-1.399748,-1.117751,-1.216612,-1.313071,-1.752392,-1.411329,-1.425142,-1.597550,-1.531932,-1.409581,-1.112324,-1.429580,661789240
145592,6307.155762,3.995435,-0.389038,-0.400602,0.289045,-0.210450,-0.271276,-0.469139,-0.479432,-0.243513,-0.399587,-0.389965,-0.420260,-0.436169,-0.586763,-0.552920,-0.400538,-0.635407,-0.461896,-0.389709,-0.194330,-0.463685,736959791
145593,6242.326172,4.165707,-0.350813,-0.368621,0.179669,-0.190526,-0.386011,-0.369994,-0.432306,-0.208131,-0.361742,-0.294581,-0.367361,-0.363063,-0.499499,-0.477914,-0.417571,-0.587380,-0.403144,-0.368972,-0.143927,-0.400654,736959817
